# Link Dataset

Dikarenakan ukuran dataset yang terlalu besar(42MB), maka untuk dataset saya upload di gdrive, berikut link Gdrive dataset:

[Belgium Population Classification](https://drive.google.com/file/d/1WGAMXm3Pkl7sp2xvdJQPbsOa4Z-MpESc/view?usp=sharing)

# Data Preprocessing

## Mengimport Library

- **'import pandas as pd'**, digunakan untuk membaca dataset csv
- **'import numpy as np'**, digunakan saat melakukan standarisasi
- **'from sklearn.model_selection import train_test_split'**, digunakan untuk membagi dataset menjadi training set dan testing set
- **'from sklearn.impute import SimpleImputer'**, digunakan saat menangani nilai null
- **'from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder'**, digunakan saat melakukan normalisasi data, StandardScaler digunakan saat melakukan standarisasi data dan OneHotEncoder digunakan saat melakukan Encoding dengan metode One Hot.

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

## Membaca file csv menggunakan encoding 'latin-1'

Encoding default membaca file csv yang digunakan pandas adalah **'UTF-8'**, mengubah encoding pandas menjadi **'latin-1'** agar file bisa dibaca oleh pandas.

In [44]:
df = pd.read_csv("BELGIUM_POPULATION_STRUCTURE_2018.csv", encoding='latin-1')

## Membuat copy dataset

Membuat variabel **'df2'** untuk menampung dataset copy yang nantinya digunakan untuk normalisasi dan standarisasi.

In [45]:
df2 = df.copy()

## Data Split

### Membagi dataset menjadi training set dan testing set dengan proporsi 70:30

Membuat variabel **'y'** untuk menampung semua column selain target/label, variabel **'x'** untuk menampung column target/label, dan variabel **'x_train, x_test, y_train, y_test'** untuk menampung semua data train dan test dengan proporsi 70% train dan 30% test.

In [46]:
x = df.drop(columns="NATIONALITY")
y = df.loc[:, "NATIONALITY"]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7)
print('Dimensi x_train', x_train.shape)
print('Dimensi x_test', x_test.shape)
print('Dimensi y_train', y_train.shape)
print('Dimensi y_test', y_test.shape)

Dimensi x_train (325792, 12)
Dimensi x_test (139626, 12)
Dimensi y_train (325792,)
Dimensi y_test (139626,)


## Data Cleaning


### Menangani nilai null

Sebelum menangani nilai null, kita perlu mengetahui apakah ada nilai null dalam dataset menggunakan **'df.isna().sum()'**.

In [47]:
df.isna().sum()

MUNICIPALITY REFNIS CODE        0
MUNICIPALITY NAME               0
DISTRICT REFNIS CODE            0
DISTRICT NAME                   0
PROVINCE REFNIS CODE        21425
PROVINCE NAME               21425
REGION REFNIS CODE              0
REGION NAME                     0
GENDER                          0
NATIONALITY                     0
CIVIL CODE STATUS               0
AGE                             0
POPULATION                      0
dtype: int64

Dari hasil diatas, diketahui ada nilai null pada column **'PROVINCE REFNIS CODE'** dan **'PROVINCE NAME'**, untuk mengatasi hal ini kita dapat menggunakan strategy median/modus dalam kasus ini saya menggunakan modus, pertama membuat variabel **'imputer'** untuk menampung fungsi **'SimpelImputer'** yang menggunakan strategy modus dengan menggunakan **'most_frequent'**.

In [48]:
imputer = SimpleImputer(strategy="most_frequent")

Melakukan penanganan nilai null menggunakan strategy modus/most_frequent pada column **'PROVINCE REFNIS CODE'** dengan menggunakan **'imputer.fit_transform(df[["PROVINCE REFNIS CODE"]])'**.

In [49]:
df["PROVINCE REFNIS CODE"] = imputer.fit_transform(df[["PROVINCE REFNIS CODE"]])

Melakukan penanganan nilai null menggunakan strategy modu/most_frequent pada column **'PROVINCE NAME'** dengan menggunakan **'imputer.fit_transform(df[["PROVINCE NAME"]])'**.

In [50]:
df["PROVINCE NAME"] = imputer.fit_transform(df[["PROVINCE NAME"]])

#### Melihat hasil penanganan nilai null.

In [51]:
df.isna().sum()

MUNICIPALITY REFNIS CODE    0
MUNICIPALITY NAME           0
DISTRICT REFNIS CODE        0
DISTRICT NAME               0
PROVINCE REFNIS CODE        0
PROVINCE NAME               0
REGION REFNIS CODE          0
REGION NAME                 0
GENDER                      0
NATIONALITY                 0
CIVIL CODE STATUS           0
AGE                         0
POPULATION                  0
dtype: int64

Dari data diatas dapat dilihat bahwa tidak ada lagi data yang bernilai null.

### Menangani Nilai Duplikat

Sebelum menangani data duplikat, kita perlu melihat apakah ada data duplikat atau tidak dengan menggunakan **'df.duplicated().sum()'**.

In [52]:
df.duplicated().sum()

0

Dikarenakan tidak ada data duplikat, untuk memenuhi ketentuan posttest, maka dilakukan penambahan data yang bernilai duplikat, dalam kasus ini saya mencoba untuk menduplikat record dengan index 43280, 54321, dan 12345 dengan menggunakan **'df = pd.concat([df, df.loc[43280]], ignore_index=True)'**, **'df = pd.concat([df, df.loc[54321]], ignore_index=True)'** dan **'df = pd.concat([df, df.loc[12345]], ignore_index=True)'**.

In [53]:
df = pd.concat([df, df.loc[43280]])
df = pd.concat([df, df.loc[54321]])
df = pd.concat([df, df.loc[12345]])

Setelah menggunakan **'concat'** akan terdapat column tambahan, maka saya gunakan **'df.iloc[:, :-1]'** untuk menghapus column tambahan, karena saya hanya membutuhkan record tambahan yang duplikat.

In [54]:
df = df.iloc[:, :-1]

Setelah menambahkan data duplikat, cek apakah data sudah terduplikat atau tidak dengan menggunakan **'df.duplicated().sum()'**.

In [55]:
df.duplicated().sum()

40

Dari hasil diatas terdapat 11 data duplikat, dengan ini kita dapat melakukan penanganan data duplikat dengan menggunakan **'df.drop_duplicates(inplace=True)'**.

In [56]:
df.drop_duplicates(inplace=True)

#### Melihat hasil penanganan nilai duplikat

Setelah melakukan penanganan data duplikat, kita bisa melakukan cek kembali apakah masih terdapat data duplikat atau tidak.

In [57]:
df.duplicated().sum()

0

Dari hasil diatas sudah tidak ada lagi data yang bernilai duplikat.

## Data Transformation

### Normalisasi

Melakukan normalisasi menggunakan **MinMaxScaler** pada salah satu attribute, dalam kasus ini saya menggunakannya pada Column **'AGE'**, membuat variabel **'age_normalized'** untuk menampung hasil normalisasi yang menggunakan **'MinMaxScaler().fit_transform(df2[["AGE"]])'** dan membuat variabel **'data'** untuk menampilkan data hasil normalisasi dalam bentuk tabel dengan menggunakan **'pd.DataFrame(age_normalized)'**.

In [58]:
age_normalized = MinMaxScaler().fit_transform(df2[["AGE"]])
data = pd.DataFrame(age_normalized)
data

,0
0,0.39
1,0.82
2,0.42
3,0.63
4,0.30
...,...
465413,0.64
465414,0.67
465415,0.74
465416,0.81


### Standarisasi

Melakukan standarisasi menggunakan **'StandardScaler()'** pada salah satu attribute, dalam kasus ini saya menggunakannya pada Column **'POPULATION'**, membuat variabel **'population_standard_scaler'** untuk menampung hasil standarisasi yang menggunakan **'StandardScaler().fit_transform(df2[["POPULATION"]])'** dan membuat variabel **'data1'** untuk menampilkan data hasil standarisasi dalam bentuk tabel dengan menggunakan **'pd.DataFrame(population_standard_scaler)'**.

In [59]:
population_standard_scaler = StandardScaler().fit_transform(df2[["POPULATION"]])
data1 = pd.DataFrame(population_standard_scaler)
data1

,0
0,0.275423
1,-0.006945
2,0.416606
3,3.961885
4,-0.163815
...,...
465413,-0.367747
465414,-0.352060
465415,-0.367747
465416,-0.367747


## Mengganti tipe data pada salah satu attribute angka

Dalam kasus ini mengubah tipe data column **'DISTRICT REFNIS CODE'** menjadi object, menggunakan **'.astype("object")'**, pertama membuat variabel **'tipe_data_object'** untuk menampung hasil perubahan tipe data, kemudian menampilkan tipe data terbaru dari column **'DISTRICT REFNIS CODE'** menggunakan **'tipe_data_object["DISTRICT REFNIS CODE"].dtypes'**

In [60]:
tipe_data_object = df[["DISTRICT REFNIS CODE"]].astype("object")
tipe_data_object["DISTRICT REFNIS CODE"].dtypes

dtype('O')

## Encoding

Melakukan One Hot Encoding pada column **'GENDER'** dengan menggunakan fungsi **'OneHotEncoder(sparse_output=False).fit_transform(df[["GENDER"]])'** dan ditampung di variabel **'gender_encoded'** dan diubah ke bentuk tabel dengan menggunakan **'pd.DataFrame(gender_encoded)'** dan gabung data One Hot ke dataset dengan menggunakan **'df.join(gender_encoded)'** pada variabel **'dataset_onehot'**

In [63]:
gender_encoded = OneHotEncoder(sparse_output=False).fit_transform(df[["GENDER"]])
gender_encoded = pd.DataFrame(gender_encoded)
dataset_oneHot = df.join(gender_encoded)
dataset_oneHot.head(10)

,MUNICIPALITY REFNIS CODE,MUNICIPALITY NAME,DISTRICT REFNIS CODE,DISTRICT NAME,PROVINCE REFNIS CODE,PROVINCE NAME,REGION REFNIS CODE,REGION NAME,GENDER,NATIONALITY,CIVIL CODE STATUS,AGE,POPULATION,0,1,2
0,71024.0,Herk-de-Stad,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,F,Belgian,Married,39.0,42.0,1.0,0.0,0.0
1,71037.0,Lummen,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,82.0,24.0,0.0,1.0,0.0
2,71011.0,Diepenbeek,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,F,Belgian,Married,42.0,51.0,1.0,0.0,0.0
3,71016.0,Genk,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,63.0,277.0,0.0,1.0,0.0
4,71017.0,Gingelom,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,F,Belgian,Married,30.0,14.0,1.0,0.0,0.0
5,71020.0,Halen,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,52.0,52.0,0.0,1.0,0.0
6,71022.0,Hasselt,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,50.0,283.0,0.0,1.0,0.0
7,71022.0,Hasselt,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,40.0,219.0,0.0,1.0,0.0
8,71045.0,Nieuwerkerken (Hasselt),71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,81.0,11.0,0.0,1.0,0.0
9,71017.0,Gingelom,71000.0,Hasselt,70000.0,Limburg,2000.0,Flemish Region,M,Belgian,Married,57.0,38.0,0.0,1.0,0.0
